In [1]:
import numpy as np

In [2]:
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.catalogs.generation.db import CatalogDBObject

In [3]:
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catUtils.baseCatalogModels import GalaxyTileObj
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.utils import ObservationMetaData



/usr/local/manual/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want

In [4]:
from lsst.sims.catUtils.baseCatalogModels import SNObj

In [5]:
s  = SNObj()

In [6]:
names = s.get_column_names()

/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geopoint'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geocenter'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geobox'
  return fn(*arg, **kw)


In [7]:
gTile = GalaxyTileObj()
class GTileCopy(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'galtileid','redshift']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e'}

In [8]:
for key in names:
    if 'snra' in names[key]:
        print key

TwinkSN


In [9]:
print names['TwinkSN']

['galtileid', 'htmid', 'snid', 't0', 'c', 'x1', 'x0', 'redshift', 'snra', 'sndec', 'cx', 'cy', 'cz']


In [10]:
class SNCopy(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'snid', 'galtileid','t0', 'x0', 'x1', 'c', 'redshift', ]
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}
                        #'x0':'%8e', 'x1':'%8e', 'c':'%8e', 't0':'%8e'}

In [11]:
degConv = np.array([1., 1./60., 1./3600.])
raConv = degConv / 24.0 * 360.
centralRA = np.dot(np.array([3., 32., 30]), raConv) #03h 32m 30s
centralDec = np.dot(np.array([-28, 6., 0.]), degConv)
patchRadius = 0.4 * np.sqrt(2) #np.dot(np.array([0.0, 10.0, 0.]), degConv)

In [12]:
TwinklesObsMetaDataSmall = ObservationMetaData(boundType='box',pointingRA=centralRA,pointingDec=centralDec,
                                          boundLength=0.167, mjd=49540.0)

In [13]:
tsmall = SNCopy(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)

In [14]:
tsmall.write_catalog('TwinklesSN.dat')

In [15]:
!wc -l TwinklesSN.dat

   72780 TwinklesSN.dat


In [16]:
!head TwinklesSN.dat

#raJ2000, decJ2000, snid, galtileid, t0, x0, x1, c, redshift
9.269085e-01, -4.856831e-01, 10000060005910006, 60005910006, 1425.8729, 2.355744e-05, 1.2825, -0.0531, 0.4431
9.269473e-01, -4.856320e-01, 60007533372, 60007533372, 1963.3116, 3.970290e-05, 0.0223, -0.0405, 0.3152
9.269600e-01, -4.856324e-01, 10000060001927699, 60001927699, 2480.8074, 1.279006e-05, -0.0907, -0.0035, 0.5018
9.270875e-01, -4.855796e-01, 50000060011104673, 60011104673, 3241.1997, 4.146627e-06, -0.8420, -0.0544, 0.7593
9.270841e-01, -4.856042e-01, 10000060002753007, 60002753007, 354.9295, 4.627642e-05, 0.4846, -0.0452, 0.2807
9.270836e-01, -4.856044e-01, 60002753007, 60002753007, 1367.0083, 5.252348e-05, 0.6385, -0.1271, 0.2807
9.270834e-01, -4.856046e-01, 20000060002753007, 60002753007, 1253.3522, 5.095395e-05, -1.0167, -0.0374, 0.2807
9.270844e-01, -4.856597e-01, 10000060004184102, 60004184102, 97.8006, 1.946555e-06, 0.3060, 0.3451, 0.6812
9.270857e-01, -4.856558e-01, 60004184102, 60004184102, 1924.5566, 4.5962

In [18]:
from lsst.sims.catUtils.mixins import TwinklesSNCat

In [19]:
ts = TwinklesSNCat(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)

RuntimeError: maximum recursion depth exceeded

In [ ]:
class SNCat()

In [21]:
np.radians(53.1029753341)

0.92682176218538093

In [25]:
gsmall = GTileCopy(db_obj=gTile, obs_metadata=TwinklesObsMetaDataSmall)

In [26]:
gsmall.write_catalog('gsmall.cat')

In [27]:
9.268184e-01

0.016176032648065813

In [28]:
np.radians(53.1027826951)

0.92681840000056304